In [87]:
# Modules
import os
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt

In [93]:
# Utils
def resize_image(self, img, label):
    img = tf.cast(img, tf.float32)
    img = tf.image.resize(img, (self.IMAGE_SIDE_SIZE, self.IMAGE_SIDE_SIZE))
    img = img / 255.0
    return img, label

def process_image_to_dataset(tensor_image):
    image = tf.io.read_file(tensor_image)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image /= 255.0
    return image

def create_data_set(images_folder):
    filenames = []
    labels = []
    
    for filename in os.listdir('./'+ images_folder):
        file_path = os.getcwd() + '/'+ images_folder +'/' + filename
        filenames.append(file_path)
        if filename.endswith("1.jpg"): 
            labels.append(0)
        else:
            labels.append(1)
            
    path_ds = tf.data.Dataset.from_tensor_slices(filenames)
    image_ds = path_ds.map(process_image_to_dataset, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(labels, tf.int64))
    dataset = tf.data.Dataset.zip((image_ds, label_ds)).prefetch(10)
    return dataset

In [67]:
# Lets Do some stuff

In [106]:
train_data = create_data_set('trainimages').shuffle(1000).batch(16)
test_data = create_data_set('testimages').shuffle(1000).batch(16)

In [113]:
base_layers = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), include_top=False)
base_layers.trainable = False
model = tf.keras.Sequential([
    base_layers,
    GlobalAveragePooling2D(),
    Dropout(0.1),
    Dense(1, activation = "sigmoid")
])

model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])

In [114]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_5 ( (None, 1280)              0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 1281      
Total params: 2,259,265
Trainable params: 1,281
Non-trainable params: 2,257,984
_________________________________________________________________


In [115]:
model.fit(train_data, epochs=10)

Epoch 1/10
75/75 [==============================] - 15s 199ms/step - loss: 0.6761 - accuracy: 0.7308
Epoch 2/10
75/75 [==============================] - 15s 201ms/step - loss: 0.6126 - accuracy: 0.9122
Epoch 3/10
75/75 [==============================] - 16s 215ms/step - loss: 0.6003 - accuracy: 0.9359
Epoch 4/10
75/75 [==============================] - 16s 214ms/step - loss: 0.5945 - accuracy: 0.9477
Epoch 5/10
75/75 [==============================] - 16s 213ms/step - loss: 0.5912 - accuracy: 0.9527
Epoch 6/10
75/75 [==============================] - 16s 215ms/step - loss: 0.5897 - accuracy: 0.9544
Epoch 7/10
75/75 [==============================] - 16s 216ms/step - loss: 0.5871 - accuracy: 0.9603
Epoch 8/10
75/75 [==============================] - 16s 214ms/step - loss: 0.5854 - accuracy: 0.9646
Epoch 9/10
75/75 [==============================] - 17s 230ms/step - loss: 0.5839 - accuracy: 0.9662
Epoch 10/10
75/75 [==============================] - 18s 239ms/step - loss: 0.5823 - accura

In [116]:
loss, acc = model.evaluate(test_data, verbose=1)
print("Model accuracy: {:5.2f}%".format(100*acc))

7/7 [==============================] - 1s 159ms/step - loss: 0.3779 - accuracy: 0.9902
Model accuracy: 99.02%


In [105]:
model.save_weights('weights.h5')